In the first step we get the unique identifiers (IDs) of the current public users from the Steam community and save these IDs to a JSON file for subsequent use. Here are the main steps:

1. **Define the `safe_request` function**:
   - This function is used to perform HTTP requests safely and has a retry mechanism for possible errors.
   - The number of retries and the delay between retries can be set.

2. **Define the `get_user_id` function**:
   - Extracts the user ID from the URL of the user's profile.
   - Extracts the user ID from the page content using a regular expression.
   - If the extraction is successful, the user ID is added to the given list of user IDs.

3. **Define the `get_all_user_ids` function**:
   - Iterates through the specified number of member list pages to get the user IDs in each page.
   - For each page, parse the HTML content to find all user profile links.
   - For each user's profile link, call the `get_user_id` function to extract the user ID and add it to the list of user IDs. 5. **Main program**: Traverse the specified number of member list pages to get the user IDs for each page.

4. **Main program**:
   - Specifies the number of member list pages to process.
   - An empty user ID list is created.
   - Calls the `get_all_user_ids` function to store the user IDs into the list.
   - Prints the total number of users found.
   - Writes the list of user IDs to a JSON file.

In [2]:
from bs4 import BeautifulSoup as bs
import re
import json
import requests
import time

In [3]:
# Get user ids
def safe_request(url, retries=3, delay=2):
    """
    Safely makes a request to the given URL with retries and delay.

    Args:
        url (str): The URL to make the request to.
        retries (int): The number of retries in case of failure.
        delay (int): The delay between retries.

    Returns:
        str: The text content of the response.
    """
    for attempt in range(retries):
        try:
            resp = requests.get(url)
            resp.raise_for_status()
            return resp.text
        except requests.exceptions.HTTPError as e:
            if resp.status_code == 502:
                print(f"Encountered HTTP 502 Bad Gateway error, retrying {attempt + 1}/{retries} in {delay} seconds...")
                time.sleep(delay)
            else:
                raise e
        except requests.exceptions.RequestException as e:
            print(f"Request failed: {e}")
    raise Exception("Reached maximum retry attempts, request failed.")

def get_user_id(user_profile, user_ids):
    """
    Extracts user ID from the user profile URL.

    Args:
        user_profile (str): The URL of the user's profile.
        user_ids (list): List to store user IDs.
    """
    try:
        page_content = safe_request(user_profile)
        for line in page_content.splitlines():
            if "steamid" in line:
                user_id = re.search(r"\"steamid\":\"(\d+)\"", line).group(1)
                if user_id:
                    user_ids.append(user_id)
                    print(f"User ID {user_id} found.")
                    break
    except Exception as e:
        print(f"Error while processing user ID: {e}")

def get_all_user_ids(member_list_page_no, user_ids):
    """
    Retrieves user IDs from multiple pages of the member list.

    Args:
        member_list_page_no (int): The total number of member list pages to process.
        user_ids (list): List to store user IDs.
    """
    for idx in range(1, member_list_page_no + 1):
        print(f"Processing Member List {idx}...")
        url = f"https://steamcommunity.com/games/steam/members?p={idx}"
        resp = requests.get(url)
        if resp.status_code == 200:
            soup = bs(resp.text, 'html.parser')
            all_users = soup.find_all("div", onclick=re.compile(r"top\.location\.href='https:\/\/steamcommunity\.com\/id\/(\w+)'"), class_=re.compile(r"online|in-game|offline"))
            for user in all_users:
                user_profile = user.div.div.div.a['href']
                get_user_id(user_profile, user_ids)
        else:
            print(f"HTTP Error {resp.status_code} while fetching member list {idx}")

if __name__ == '__main__':
    member_list_page_no = 342
    user_ids = []
    get_all_user_ids(member_list_page_no, user_ids)
    print(f"Total users found: {len(user_ids)}")
    with open('/Users/a1234/Desktop/workspace/Steam_Recommendation_System_777_tp/Data/user_ids.json', 'w') as f:
        json.dump(user_ids, f)
    print("User IDs dumped to file: Data/user_ids.json")

Processing Member List 1...
User ID 76561197960434622 found.
User ID 76561197972495328 found.
User ID 76561198028573551 found.
User ID 76561197970323416 found.
User ID 76561198007695232 found.
User ID 76561197970565175 found.
User ID 76561197968459473 found.
User ID 76561198024187698 found.
User ID 76561197980258575 found.
User ID 76561198000175595 found.
User ID 76561198015158492 found.
User ID 76561197993258642 found.
User ID 76561197972978861 found.
User ID 76561198106631362 found.
User ID 76561198150708893 found.
User ID 76561198059745288 found.
User ID 76561198030442423 found.
User ID 76561197993715298 found.
User ID 76561198077131365 found.
User ID 76561198278664273 found.
User ID 76561198072796930 found.
User ID 76561198102302108 found.
User ID 76561197990067826 found.
User ID 76561198076780091 found.
User ID 76561198109429966 found.
User ID 76561198128232705 found.
User ID 76561198067064616 found.
User ID 76561197972179036 found.
User ID 76561198089451726 found.
User ID 7656119